<a href="https://colab.research.google.com/github/HussamSoufi/googleColab/blob/main/BlindAI_st_gcn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Example of BlindAI deployment with st-gcn

We will see in this notebook how to deploy and query a st-gcn model on BlindAI Cloud. To upload of the model, make sure you have an API key.

You can get one on the [Mithril Cloud](https://cloud.mithrilsecurity.io/).

First, let's install `blindai`.

In [5]:
!pip install blindai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Uploading the model to BlindAI

The first step is to get the model in ONNX format. Let's pull the st-gcn model and export in in ONNX.

In [6]:
# This is a fork of https://github.com/yysijie/st-gcn
!wget -N --quiet --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1EsbOCzY02VZZvGlbCS5E6jpm9TfjUwJp' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1EsbOCzY02VZZvGlbCS5E6jpm9TfjUwJp" -O st-gcn-onnx.zip && rm -rf /tmp/cookies.txt
!unzip -o st-gcn-onnx.zip

Archive:  st-gcn-onnx.zip
f1182f4580e173dde4759341d438bad37722ef2d
   creating: st-gcn-onnx-master/
  inflating: st-gcn-onnx-master/.gitignore  
  inflating: st-gcn-onnx-master/ISSUE_TEMPLATE.md  
  inflating: st-gcn-onnx-master/LICENSE  
  inflating: st-gcn-onnx-master/OLD_README.md  
  inflating: st-gcn-onnx-master/README.md  
   creating: st-gcn-onnx-master/config/
   creating: st-gcn-onnx-master/config/st_gcn.twostream/
   creating: st-gcn-onnx-master/config/st_gcn.twostream/ntu-xsub/
  inflating: st-gcn-onnx-master/config/st_gcn.twostream/ntu-xsub/train.yaml  
   creating: st-gcn-onnx-master/config/st_gcn.twostream/ntu-xview/
  inflating: st-gcn-onnx-master/config/st_gcn.twostream/ntu-xview/train.yaml  
   creating: st-gcn-onnx-master/config/st_gcn/
   creating: st-gcn-onnx-master/config/st_gcn/kinetics-skeleton/
  inflating: st-gcn-onnx-master/config/st_gcn/kinetics-skeleton/demo_offline.yaml  
  inflating: st-gcn-onnx-master/config/st_gcn/kinetics-skeleton/demo_old.yaml  
  infl

In [7]:
import sys
sys.path.append('/content/st-gcn-onnx-master')

from net.st_gcn import Model
import torch

mod = Model(in_channels=3, num_class=400, edge_importance_weighting=True, graph_args={"layout": "openpose", "strategy": "spatial"})

inp = torch.randn(1, 3, 300, 18, 2, dtype=torch.float32)

torch.onnx.export(mod, inp, "st-gcn.onnx", input_names=["input"], output_names=["output"], opset_version=13)

/content/st-gcn-onnx-master/net/utils/tgcn.py:58: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert A.size(0) == self.kernel_size
/content/st-gcn-onnx-master/net/utils/tgcn.py:63: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  x = x.view(n, self.kernel_size, kc//self.kernel_size, t, v)
/content/st-gcn-onnx-master/net/st_gcn.py:87: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record th

In [11]:
import torch
import numpy as np


inp = torch.from_numpy(np.load("./out.npy"))
inp

tensor([[[[[ 0.0000,  0.0000],
           [ 0.0000,  0.0000],
           [ 0.0000,  0.0000],
           ...,
           [ 0.0000,  0.0000],
           [ 0.0000,  0.0000],
           [ 0.0000,  0.0000]],

          [[ 0.1280, -0.3470],
           [ 0.1120, -0.3610],
           [ 0.0950, -0.3610],
           ...,
           [ 0.1280,  0.0000],
           [ 0.1120, -0.3550],
           [ 0.0000,  0.0000]],

          [[ 0.1260, -0.3530],
           [ 0.1120, -0.3610],
           [ 0.0950, -0.3630],
           ...,
           [ 0.1280,  0.0000],
           [ 0.1120, -0.3610],
           [ 0.1180,  0.0000]],

          ...,

          [[ 0.0000,  0.0000],
           [ 0.0000,  0.0000],
           [ 0.0000,  0.0000],
           ...,
           [ 0.0000,  0.0000],
           [ 0.0000,  0.0000],
           [ 0.0000,  0.0000]],

          [[ 0.0000,  0.0000],
           [ 0.0000,  0.0000],
           [ 0.0000,  0.0000],
           ...,
           [ 0.0000,  0.0000],
           [ 0.0000,  0.0000

Now we can upload the model to BlindAI Cloud. To upload of the model, make sure you have an API key.

You can get one on the [Mithril Cloud](https://cloud.mithrilsecurity.io/).

You might get an error if the name you want to use is already taken, as models are uniquely identified by their `model_id`. We will implement namespace soon to avoid that. Meanwhile, you will have to choose a unique ID.

In [8]:
import blindai
import uuid

api_key = "7bb41096c82a212815e21bc324f89813" # Enter your API key here
model_id = "st-gcn-" + str(uuid.uuid4())

# Upload the ONNX file to the remote enclave
with blindai.Connection(api_key=api_key) as client:
    response = client.upload_model("st-gcn.onnx", model_id=model_id)

## Querying the model

Now we just need to connect to the model inside the secure enclave and query it.

In [12]:
import blindai
import torch

with blindai.Connection(api_key=api_key) as client:
  # Send data to the st-gcn model
  prediction = client.predict(model_id, inp)
  print(prediction.inference_time)

525
